# Students Performance Analysis by Region
This notebook analyzes student performance by region.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load CSV file
df = pd.read_csv(r"C:/Users/DELL/Downloads/ethiopian_students_dataset.csv")

# View first 5 rows
print(df.head())

# Access a column
print(df.columns)

In [ ]:
# 1. Get the value counts of dtypes
dtype_counts = df.dtypes.value_counts().reset_index()
dtype_counts.columns = ['Data Type', 'Count']

# 2. Plotting with the fix
plt.figure(figsize=(10, 6))

# Fix: Assign 'Data Type' to hue and set legend=False
sns.barplot(
    data=dtype_counts,
    x='Data Type',
    y='Count',
    hue='Data Type',
    palette='viridis',
    legend=False
)

plt.title('Distribution of Data Types in Student Dataset', fontsize=14)
plt.ylabel('Number of Columns')
plt.xlabel('Data Type')

# 3. Add labels on top of bars
for i, count in enumerate(dtype_counts['Count']):
    plt.text(i, count + 5, str(count), ha='center', fontweight='bold')

plt.show()

In [ ]:
# ================================
# 1️⃣ INITIAL CLEANING & ENCODING
# ================================
# Drop Student_ID (never used in ML)
df = df.drop(columns=['Student_ID'], errors='ignore')

# Encode Field_Choice (Social=0, Natural=1)
df['Field_Choice'] = df['Field_Choice'].map({'Social': 0, 'Natural': 1})

# Fill missing Career_Interest with "Unknown"
df['Career_Interest'] = df['Career_Interest'].fillna('Unknown')


# ================================
# 2️⃣ DEFINE EDUCATION STAGES
# ================================
lower_primary = ['Grade_1', 'Grade_2', 'Grade_3', 'Grade_4']
upper_primary = ['Grade_5', 'Grade_6', 'Grade_7', 'Grade_8']
secondary     = ['Grade_9', 'Grade_10']
preparatory   = ['Grade_11', 'Grade_12']

stages = {
    'Lower_Primary': lower_primary,
    'Upper_Primary': upper_primary,
    'Secondary': secondary,
    'Preparatory': preparatory
}


# ================================
# 3️⃣ HELPER FUNCTION TO AGGREGATE GRADES
# ================================
def stage_average(df, grades, metric_keywords):
    """
    Compute average across all columns for a given stage and metric keywords.
    Returns the average series and list of original columns used.
    """
    cols = []
    for g in grades:
        for keyword in metric_keywords:
            cols += [c for c in df.columns if c.startswith(g) and keyword.lower() in c.lower()]
    cols = list(set(cols))
    return df[cols].mean(axis=1), cols


# ================================
# 4️⃣ AGGREGATE TEST SCORE, ATTENDANCE, HW, PARTICIPATION
# ================================
metrics_dict = {
    'Test_Score': ['Test_Score'],
    'Attendance': ['Attendance'],
    'HW_Completion': ['Homework_Completion'],
    'Participation': ['Participation']
}

cols_to_drop = []

for metric_name, keywords in metrics_dict.items():
    for stage_name, grades in stages.items():
        col_name = f'Avg_{metric_name}_{stage_name}'
        df[col_name], original_cols = stage_average(df, grades, keywords)
        cols_to_drop += original_cols

# Drop original grade-level columns
df.drop(columns=list(set(cols_to_drop)), inplace=True)

# Columns list for display
aggregated_cols = [f'Avg_{m}_{s}' for m in metrics_dict.keys() for s in stages.keys()]
print("Aggregated averages per Education Stage (head):")
print(df[aggregated_cols].head())


# ================================
# 5️⃣ AGGREGATE TEXTBOOK ACCESS
# ================================
# Convert Yes/No → 1/0 safely
textbook_cols = [c for c in df.columns if 'Textbook' in c]
for col in textbook_cols:
    df[col] = df[col].replace({'Yes': 1, 'No': 0}).infer_objects(copy=False)

# Helper function for textbook access per stage
def textbook_access(df, grade_prefixes):
    cols = []
    for g in grade_prefixes:
        cols.extend([c for c in df.columns if c.startswith(g) and 'Textbook' in c])
    return df[cols].mean(axis=1) if len(cols) > 0 else pd.Series(0, index=df.index)

# Create aggregated textbook access per stage
new_cols_df = pd.DataFrame({
    'Textbook_Access_1_4': textbook_access(df, lower_primary),
    'Textbook_Access_5_8': textbook_access(df, upper_primary),
    'Textbook_Access_9_10': textbook_access(df, secondary),
    'Textbook_Access_11_12': textbook_access(df, preparatory)
})

df = pd.concat([df, new_cols_df], axis=1)
df = df.loc[:, ~df.columns.duplicated()]  # remove duplicates

# Display and visualize
textbook_summary_cols = [c for c in new_cols_df.columns if c in df.columns]
print(df[textbook_summary_cols].head())

plt.figure(figsize=(10, 6))
sns.boxplot(data=df[textbook_summary_cols])
plt.title('Textbook Access Distribution by Education Level', fontsize=14, fontweight='bold')
plt.ylabel('Access Score (0 to 1)')
plt.xticks(rotation=15)
plt.grid(alpha=0.3)
plt.show()


# ================================
# 6️⃣ TRACK-BASED NATIONAL EXAMS
# ================================
# Subjects per track
social_subjects = ['National_Exam_History', 'National_Exam_Geography',
                   'National_Exam_Economics', 'National_Exam_Math_Social']

natural_subjects = ['National_Exam_Biology', 'National_Exam_Chemistry',
                    'National_Exam_Physics', 'National_Exam_Math_Natural']

# Track-specific averages
df['Social_Track_Subject_Avg']  = df[social_subjects].mean(axis=1)
df['Natural_Track_Subject_Avg'] = df[natural_subjects].mean(axis=1)

# Track-based assignment
df['Track_Subject_Average'] = np.where(
    df['Field_Choice'] == 0,
    df['Social_Track_Subject_Avg'],
    df['Natural_Track_Subject_Avg']
)

# Common subjects for all students
common_subjects = ['National_Exam_Aptitude', 'National_Exam_English',
                   'National_Exam_Civics_and_Ethical_Education']
df['Common_Exam_Average'] = df[common_subjects].mean(axis=1)

# Overall Track Exam Average
df['Track_Exam_Average'] = (df['Common_Exam_Average'] + df['Track_Subject_Average']) / 2

# Display new exam columns
exam_cols = [
    'Social_Track_Subject_Avg',
    'Natural_Track_Subject_Avg',
    'Track_Subject_Average',
    'Common_Exam_Average',
    'Track_Exam_Average'
]
print("New Aggregated National Exam Features:")
print(df[exam_cols].head())


# ================================
# 7️⃣ VISUALIZATION: Exam Scores
# ================================
sns.set_theme(style="whitegrid")
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Boxplot: Common vs Track vs Overall
sns.boxplot(data=df[['Common_Exam_Average', 'Track_Subject_Average', 'Track_Exam_Average']],
            ax=axes[0], palette="Set2")
axes[0].set_title('Distribution of Aggregate Exam Scores')
axes[0].set_ylabel('Score (0-100)')

# KDE: Track Exam Average by Field Choice
for choice, label in [(0, 'Social Science'), (1, 'Natural Science')]:
    subset = df[df['Field_Choice'] == choice]
    sns.kdeplot(subset['Track_Exam_Average'], ax=axes[1], label=label, fill=True)

axes[1].set_title('Track Exam Average: Social vs. Natural')
axes[1].set_xlabel('Score')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# DROP ORIGINAL HIGH-DIMENSION COLUMNS
drop_cols = [c for c in df.columns if c.startswith('Grade_')]
drop_cols += [c for c in df.columns if c.startswith('National_Exam_')]

df = df.drop(columns=drop_cols)
# -------------------------------
# 0️⃣ Drop leaking exam average columns
# -------------------------------
leak_cols = [
    'Social_Track_Subject_Avg',
    'Natural_Track_Subject_Avg',
    'Track_Exam_Average',
    'Track_Subject_Average',
    'Common_Exam_Average',
     'School_ID']

df = df.drop(columns=[c for c in leak_cols if c in df.columns])

# fix null value
df['Health_Issue'] = df['Health_Issue'].fillna('No Issue')
df['Father_Education'] = df['Father_Education'].fillna('Unknown')
df['Mother_Education'] = df['Mother_Education'].fillna('Unknown')

# FINAL CHECK
print(df.shape)
print(df.head())
print("all columns:",df.columns)

# Find duplicates
duplicates = df[df.duplicated()]

In [ ]:
# ================================
# ALL-IN-ONE CATEGORICAL ENCODING
# ================================
# -------------------------------
# 1️⃣ Fill missing / fix NaNs
# -------------------------------
if 'Health_Issue' in df.columns:
    df['Health_Issue'] = df['Health_Issue'].fillna('No Issue')

for col in ['Father_Education', 'Mother_Education']:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')

# -------------------------------
# 2️⃣ Binary encoding
# -------------------------------
binary_maps = {
    'Gender': {'Male': 0, 'Female': 1},
    'Home_Internet_Access': {'No': 0, 'Yes': 1},
    'Electricity_Access': {'No': 0, 'Yes': 1},
    'School_Location': {'Rural': 0, 'Urban': 1}
}

for col, mapping in binary_maps.items():
    if col in df.columns:
        df[col] = df[col].map(mapping)

# -------------------------------
# 3️⃣ Ordinal encoding (Parents Education)
# -------------------------------
edu_map = {'Unknown': 0, 'Primary': 1, 'High School': 2, 'College': 3, 'University': 4}
for col in ['Father_Education', 'Mother_Education']:
    enc_col = col + '_Encoded'
    if col in df.columns:
        df[enc_col] = df[col].map(edu_map)
        df.drop(columns=[col], inplace=True)

# -------------------------------
# 4️⃣ One-Hot Encoding (moderate cardinality)
# -------------------------------
ohe_cols = [c for c in [ 'School_Type', 'Health_Issue'] if c in df.columns]
if ohe_cols:
    df = pd.get_dummies(df, columns=ohe_cols, drop_first=True)

# -------------------------------
# Convert Date_of_Birth → Age (numeric)
# -------------------------------
CURRENT_DATE = pd.Timestamp('2026-01-30')  # fixed date for reproducibility

if 'Date_of_Birth' in df.columns:
    df['Date_of_Birth'] = pd.to_datetime(df['Date_of_Birth'], errors='coerce')
    df['Age'] = ((CURRENT_DATE - df['Date_of_Birth']).dt.days // 365).astype(float)
    df.drop(columns=['Date_of_Birth'], inplace=True)

# -------------------------------
# 6️⃣ Safety check
# -------------------------------
print("Categorical encoding completed.")
print("Columns now:", df.select_dtypes(include='object').columns.tolist())  # should be empty

In [ ]:
# -------------------------------
# 🔟 Drop Raw Categorical Columns
# -------------------------------
drop_cols = [
    'Father_Education', 'Mother_Education','Career_Interest',
    'Health_Issue', 'Date_of_Birth',
    'School_ID', 'School_Type',
]
df.drop(columns=[c for c in drop_cols if c in df.columns], inplace=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

cluster_df = df[cluster_features]
#2️⃣ Compute correlation matrix
corr_matrix = cluster_df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Correlation Matrix - Selected Cluster Features", fontsize=16)
plt.show()

In [ ]:
# -----------------------------
# Helper function: get grade-level columns safely due to high VIF of features
# -----------------------------
def get_grade_columns(df, keywords):
    """
    Returns a DataFrame of all columns containing any of the keywords.
    If no columns found, returns a dummy column of zeros to avoid errors.
    """
    cols = [c for c in df.columns if any(k in c for k in keywords)]
    if not cols:
        return pd.DataFrame(0, index=df.index, columns=['dummy'])
    return df[cols]

# -----------------------------
# Create composite features
# -----------------------------
df['Overall_Test_Score_Avg'] = df[['Avg_Test_Score_Preparatory',
                                   'Avg_Test_Score_Lower_Primary',
                                   'Avg_Test_Score_Upper_Primary',
                                   'Avg_Test_Score_Secondary']].mean(axis=1)

df['Overall_Textbook_Access_Composite'] = df[['Textbook_Access_1_4', 'Textbook_Access_5_8',
                                      'Textbook_Access_9_10', 'Textbook_Access_11_12']].mean(axis=1)

df['Academic_Growth_Primary_to_Secondary'] = df['Avg_Test_Score_Secondary'] - df['Avg_Test_Score_Lower_Primary']

# -----------------------------
# Overall engagement averages
# -----------------------------
# -----------------------------
# Create overall averages (explicit columns)
# -----------------------------

# Attendance columns
attendance_cols = [
    'Avg_Attendance_Lower_Primary',
    'Avg_Attendance_Upper_Primary',
    'Avg_Attendance_Secondary',
    'Avg_Attendance_Preparatory'
]

df['Overall_Avg_Attendance'] = df[attendance_cols].mean(axis=1)

# Homework columns
homework_cols = [
    'Avg_HW_Completion_Lower_Primary',
    'Avg_HW_Completion_Upper_Primary',
    'Avg_HW_Completion_Secondary',
    'Avg_HW_Completion_Preparatory'
]

df['Overall_Avg_Homework'] = df[homework_cols].mean(axis=1)

# Participation columns
participation_cols = [
    'Avg_Participation_Lower_Primary',
    'Avg_Participation_Upper_Primary',
    'Avg_Participation_Secondary',
    'Avg_Participation_Preparatory'
]

df['Overall_Avg_Participation'] = df[participation_cols].mean(axis=1)

# -----------------------------
# Composite engagement score (weighted)
# -----------------------------
df['Overall_Engagement_Score'] = (
    df['Overall_Avg_Attendance'] * 0.4 +
    df['Overall_Avg_Homework'] * 0.3 +
    df['Overall_Avg_Participation'] * 0.3
)

# -----------------------------
# Quick overview
# -----------------------------
print("\nCreated overall averages:")
for col in ['Overall_Avg_Attendance', 'Overall_Avg_Homework','Overall_Textbook_Access_Composite', 'Overall_Avg_Participation', 'Overall_Engagement_Score']:
    print(f"- {col}: {df[col].mean():.2f}")

In [ ]:
# DROP ORIGINAL HIGH-DIMENSION COLUMNS

drop_cols = []

# Test Scores
drop_cols += [c for c in df.columns if c.startswith('Avg_Test_Score_')]

# Textbook Access
drop_cols += [c for c in df.columns if c.startswith('Textbook_Access_')]

# Attendance, Participation, Homework
drop_cols += [c for c in df.columns if c.startswith('Avg_Attendance_')]
drop_cols += [c for c in df.columns if c.startswith('Avg_Participation_')]
drop_cols += [c for c in df.columns if c.startswith('Avg_HW_Completion_')]

# Drop safely
df = df.drop(columns=drop_cols, errors='ignore')

print(f"Dropped {len(drop_cols)} original aggregated columns.")

In [ ]:
#=======================================================
# Not Remove rows with any outlier becuase it is necesary
#=======================================================
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
# Calculate Q1, Q3, and IQR
Q1 = df[numeric_cols].quantile(0.25)
Q3 = df[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

# Detect outliers
outliers = ((df[numeric_cols] < (Q1 - 1.5 * IQR)) | (df[numeric_cols] > (Q3 + 1.5 * IQR)))

# Count of outliers per column
outlier_counts = outliers.sum()
print("Outliers per column:\n", outlier_counts)


In [ ]:
# ==============================================
# COMPLETE STUDENT PERFORMANCE CLUSTERING PIPELINE
# ==============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

# ----------------------------
# 1️⃣ Define clustering features
# ----------------------------
cluster_features = [
    'Total_National_Exam_Score',
    'Overall_Test_Score_Avg',
    'Overall_Avg_Attendance',
    'Overall_Avg_Homework',
    'Overall_Avg_Participation',
    'Overall_Engagement_Score',
    'Overall_Textbook_Access_Composite'
    'School_Resources_Score',
    'School_Academic_Score',
    'Teacher_Student_Ratio',
    'Student_to_Resources_Ratio',
    'Parental_Involvement',
    'School_Location',
]

# ----------------------------
# 2️⃣ Ensure boolean columns are numeric
# ----------------------------
bool_cols = ['School_Type_NGO-operated', 'School_Type_Private', 'School_Type_Public']
for col in bool_cols:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype(int)

# ----------------------------
# 3️⃣ Standardize features
# ----------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[cluster_features])

# ----------------------------
# 4️⃣ Run MiniBatchKMeans
# ----------------------------
optimal_k = 3
kmeans = MiniBatchKMeans(n_clusters=optimal_k, batch_size=8192, random_state=42)
df['Performance_Cluster'] = kmeans.fit_predict(X_scaled)

# ----------------------------
# 5️⃣ Map clusters to labels
# High = highest exam scores, Low = lowest
# ----------------------------
cluster_order = (
    df.groupby('Performance_Cluster')['Total_National_Exam_Score']
      .mean()
      .sort_values(ascending=False)
      .index
)

label_map = {
    cluster_order[0]: 'High',
    cluster_order[1]: 'Medium',
    cluster_order[2]: 'Low'
}

df['Performance_Label'] = df['Performance_Cluster'].map(label_map)

print("\n🏷️ CLUSTER → LABEL MAPPING:")
for k, v in label_map.items():
    print(f"Cluster {k} → {v}")

# ----------------------------
# 6️⃣ Cluster sizes
# ----------------------------
print("\n📦 CLUSTER SIZES:\n")
print(df['Performance_Label'].value_counts())

# ----------------------------
# 7️⃣ Compute Silhouette Score
# ----------------------------
sil_score = silhouette_score(X_scaled, df['Performance_Cluster'])
print(f"\n✅ Silhouette Score for {optimal_k} clusters: {sil_score:.4f}")

# ----------------------------
# 8️⃣ Cluster profiles
# ----------------------------
cluster_profile = df.groupby('Performance_Cluster')[cluster_features].mean()
print("\nCLUSTER PROFILES:\n")
print(cluster_profile)

# ----------------------------
# 9️⃣ Cluster sizes (raw)
# ----------------------------
cluster_sizes = df['Performance_Cluster'].value_counts().sort_index()
print("\nCLUSTER SIZES (RAW):\n")
print(cluster_sizes)

# ----------------------------
# 🔹 10️⃣ Regional Distribution of Clusters
# ----------------------------
if 'Region' in df.columns:
    region_cluster_pct = (
        df.groupby(['Region', 'Performance_Cluster'])
          .size()
          .groupby(level=0)
          .apply(lambda x: x / x.sum())
          .unstack()
    )

    plt.figure(figsize=(12,6))
    sns.heatmap(region_cluster_pct, annot=True, fmt=".2f", cmap="Blues")
    plt.title("Regional Distribution of Performance Clusters")
    plt.show()

# ----------------------------
# 🔹 11️⃣ Regional Risk Map (% Low Performance)
# ----------------------------
if 'Region' in df.columns:
    regional_risk = (
        df.groupby('Region')['Performance_Label']
          .apply(lambda x: (x == 'Low').mean() * 100)
          .sort_values(ascending=False)
    )

    print("\n🗺️ REGIONAL RISK (% LOW PERFORMANCE):\n")
    print(regional_risk)

    plt.figure(figsize=(12,6))
    sns.barplot(
        x=regional_risk.index,
        y=regional_risk.values,
        color='brown'
    )
    plt.xticks(rotation=45, ha='right')
    plt.ylabel("Low Performance (%)")
    plt.title("Regional Risk: % of Low-Performing Students")
    plt.show()

# ----------------------------
# 🔹 12️⃣ PCA Visualization of clusters
# ----------------------------
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(8,6))
sns.scatterplot(
    x=X_pca[:,0],
    y=X_pca[:,1],
    hue=df['Performance_Label'],
    palette={'High':'green','Medium':'orange','Low':'red'},
    alpha=0.6
)
plt.title("PCA View of Students (Full Dataset)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.show()

print("\n✅ Clustering + labeling aligned correctly.")